In [ ]:
import os
import sys
from hydromt_sfincs.sfincs import Sfincs

Create grid from scratch

In [ ]:
sf = Sfincs()

In [ ]:
sf.inp.x0 = 268650
sf.inp.y0 = 5018550
sf.inp.dx = 150
sf.inp.dy = 150
sf.inp.nmax = 272  
sf.inp.mmax = 425 
sf.inp.rotation = 0
sf.inp.crs = 32633

sf.create_grid(grid_type="regular")

Create mask purely based on include- and exclude polygons

In [ ]:
from misc import tekal as tek #should we move/copy the misc towards hydromt-sfincs?
import numpy as np
from shapely.geometry import MultiPolygon, Polygon
import geopandas as gpd

def tekal_to_gdf(file_name, crs, zmin=None, zmax=None) -> gpd.GeoDataFrame:
    polygons = []

    D = tek.tekal(file_name)
    D.info()
    for j in range(len(D.blocks)):
        m=D.read(j)
        x = m[0,:,0]
        y = m[1,:,0]
        data = np.vstack([x,y])
        polygons.append(Polygon(data.T))

    gdf = gpd.GeoDataFrame(crs=crs, geometry=polygons)
    #TODO: add functionality to scripts if wanted
    gdf["zmin"] = zmin 
    gdf["zmax"] = zmax 

    return gdf

In [ ]:
file_name = "sfincs_coastal//include.pol"
gdf_include = tekal_to_gdf(file_name=file_name, crs='epsg:'+str(sf.inp.crs))


In [ ]:
file_name = "sfincs_coastal//exclude.pol"
gdf_exclude = tekal_to_gdf(file_name=file_name, crs='epsg:'+str(sf.inp.crs))

In [ ]:
# multipolygon_geom = MultiPolygon(polygons)
# gdf2 = gpd.GeoDataFrame(crs='epsg:'+str(sf.inp.crs), geometry=[multipolygon_geom])

In [ ]:
mask = sf.grid.create_mask_active(gdf_include=gdf_include, gdf_exclude=gdf_exclude)
mask.plot.imshow()
# sf.grid.mask.plot.imshow()

Also take into account depth

In [ ]:
from hydromt import DataCatalog
from hydromt_sfincs import workflows
data_catalog = DataCatalog() # north Italy

# sf.grid.data.raster.set_crs(sf.grid.crs)
region = sf.grid.mask.raster.box
da_dep= data_catalog.get_rasterdataset(
    'gebco', variables=['elevtn'], geom=region, buffer=5
)

# reproject to destination CRS
check_crs = sf.grid.crs is not None and da_dep.raster.crs != sf.grid.crs
check_res = abs(da_dep.raster.res[0]) != sf.grid.dx
if check_crs or check_res:
    da_dep = da_dep.raster.reproject(
        dst_res=sf.grid.dx, dst_crs=sf.grid.crs, align=True, method="bilinear"
    )
# clip & mask
da_dep = (
    da_dep.raster.clip_geom(geom=region, mask=True)
    .raster.mask_nodata() #set nodata to nan
    .fillna(-9999)  # force nodata value to be -9999
    .round(2)  # cm precision
)
da_dep.raster.set_nodata(-9999)

# eventually merge datasets
# da_dep2= data_catalog.get_rasterdataset(
#     'merit_hydro', variables=['elevtn'], geom=region, buffer=5
# )
# da_dep_merged = workflows.merge_topobathy(
#     da_dep,
#     da_dep2,
# )

# da_dep_merged.raster.mask_nodata().plot.imshow()


In [ ]:
# assign to SFINCS model
da_dep.name = "dep"

# NOTE: mask cannot be named mask since mask is in coordinates of xarray -> changed into msk

sf.grid.data.update(da_dep.to_dataset())

In [ ]:
#play around with include, drop_area and fill_area
mask = sf.grid.create_mask_active(elv_min=-5, elv_max=10, fill_area = 10, drop_area = 10, gdf_include=gdf_include)
mask.plot.imshow()

Mask boundaries

In [ ]:
file_name = "sfincs_coastal\\bnd_wl.pol"
gdf_wl = tekal_to_gdf(file_name=file_name, crs='epsg:'+str(sf.inp.crs))

file_name = "sfincs_coastal\\bnd_out.pol"
gdf_outflow = tekal_to_gdf(file_name=file_name, crs='epsg:'+str(sf.inp.crs))

In [ ]:
sf.grid.create_mask_bounds(btype="waterlevel",gdf_include=gdf_wl, elv_max = -2)
sf.grid.create_mask_bounds(btype="outflow",gdf_include=gdf_outflow, elv_min = 2)

sf.grid.mask.plot.imshow()